In [ ]:
from owlready2 import get_ontology, Restriction, owlready, Or
from parso.python.tree import Class

In [ ]:
url = "https://raw.githubusercontent.com/Food-Ninja/PouringFood/main/owl/PouringFood.owl"
onto = get_ontology(url).load()
SOMA = onto.get_namespace("http://www.ease-crc.org/ont/SOMA.owl#")
POUR = onto.get_namespace("http://www.ease-crc.org/ont/pouring_food#")
DUL = onto.get_namespace("http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#")
OBO = onto.get_namespace("http://purl.obolibrary.org/obo/")
QUDT = onto.get_namespace("http://qudt.org/schema/qudt#")


In [ ]:
def whattool(task):
    for parent in task.is_a:
        if isinstance(parent, Restriction) and parent.property == DUL.hasParticipant:
            return parent.value

In [ ]:
def get_consistency(food_cls):
    for parent in food_cls.is_a:
        if isinstance(parent, Restriction):
            if parent.property == POUR.hasConsistency:
                return parent.value
        
    for parent in food_cls.is_a:
        for child in parent.is_a:
            if isinstance(child, Restriction):
                if child.property == POUR.hasConsistency:
                    return child.value

In [ ]:
def get_angle_class_from_consistency(consistency):
    consistencies = []
    for subclass in POUR.PouringAngle.subclasses():
        for parent in subclass.is_a:
            if not isinstance(parent, Restriction):
               continue
            if parent.property == POUR.hasInputObject:
                for child in parent.value.Classes:                
                    if isinstance(child, Restriction):
                        if isinstance(child.value, Or):
                            for i in child.value.Classes:
                                consistencies.append(i)
                        consistencies.append(child.value)
                        if consistency in consistencies:
                            consistencies.clear()
                            return subclass
                    

In [ ]:
def get_angles(food_cls):
    angles = []
    if isinstance(get_consistency(food_cls), Or):
        
        if POUR.Liquid in get_consistency(food_cls).Classes:
            for parent in get_angle_class_from_consistency(POUR.Liquid).is_a:
                if isinstance(parent, Restriction):
                    if parent.property == QUDT.valueQuantity:
                        angles.append(parent.cardinality)
            
            for parent in get_angle_class_from_consistency(POUR.Moist).is_a:
                if isinstance(parent, Restriction):
                    if parent.property == QUDT.valueQuantity:
                        angles.append(parent.cardinality)
    
        if POUR.Liquid not in get_consistency(food_cls).Classes:
                for parent in get_angle_class_from_consistency(get_consistency(food_cls).Classes[0]).is_a:
                    if isinstance(parent, Restriction):
                        if parent.property == QUDT.valueQuantity:
                            angles.append(parent.cardinality)
    
    else:        
        for parent in get_angle_class_from_consistency(get_consistency(food_cls)).is_a:
            if isinstance(parent, Restriction):
                if parent.property == QUDT.valueQuantity:
                    angles.append(parent.cardinality)
                
                
    
    min_angle = min(angles)
    max_angle = max(angles)
    
    return min_angle, max_angle

In [ ]:
def get_duration_class_from_consistency(consistency):
    consistencies = []
    for subclass in POUR.PouringDuration.subclasses():
        for parent in subclass.is_a:
            if not isinstance(parent, Restriction):
               continue
            if parent.property == POUR.hasInputObject:
                for child in parent.value.Classes:                
                    if isinstance(child, Restriction):
                        if isinstance(child.value, Or):
                            for i in child.value.Classes:
                                consistencies.append(i)
                        consistencies.append(child.value)
                        if consistency in consistencies:
                            consistencies.clear()
                            return subclass



In [ ]:
def get_durations(food_cls):
    durations = []
    
    if isinstance(get_consistency(food_cls), Or):
        print("here")
        if POUR.Liquid in get_consistency(food_cls).Classes:
            
            for parent in get_duration_class_from_consistency(POUR.Liquid).is_a:
                if isinstance(parent, Restriction):
                    if parent.property == QUDT.valueQuantity:
                        durations.append(parent.cardinality)
            
            for parent in get_angle_class_from_consistency(POUR.Moist).is_a:
                if isinstance(parent, Restriction):
                    if parent.property == QUDT.valueQuantity:
                        durations.append(parent.cardinality)
        
        if POUR.Liquid not in get_consistency(food_cls).Classes:
            for parent in get_duration_class_from_consistency(get_consistency(food_cls).Classes[0]).is_a:
                if isinstance(parent, Restriction):
                    if parent.property == QUDT.valueQuantity:
                        durations.append(parent.cardinality)
    else:
        for parent in get_duration_class_from_consistency(get_consistency(food_cls)).is_a:
            if isinstance(parent, Restriction):
                if parent.property == QUDT.valueQuantity:
                    durations.append(parent.cardinality)
    
    min_duration = min(durations)
    max_duration = max(durations)
    
    return min_duration, max_duration

get_durations(OBO.FOODON_03306347)

In [ ]:
# Pipeline
task = POUR.Draining
Food_object = POUR.marinade
print("Input Task: " + str(task))
print("Input Food Object: " + str(Food_object))
print("Needed Tool:")
print(whattool(task))
print("Pouring Angles:")
print(get_angles(food_cls=Food_object))
print("Durations:")
print(get_durations(food_cls=Food_object))

# Notes
- ToDo: Food Klassen "Marinade", "Pasta", "Rice" haben eine Verorderung in der Consistency: Dry or Liquid, Dry or Moist -> Wie berücksichtigt man das?
- 